In [1]:
import string
import random
import re
import numpy as np

from collections import Counter

from nltk import everygrams

In [2]:
with open('data/WarAndPeace.txt') as f:
    text = f.read()

In [3]:
rus_text = re.compile(r'[^а-я ]+')
spaces = re.compile(r'\s+')

In [4]:
def preprocess_text(text):
    return spaces.subn(' ', rus_text.subn('', text.lower() + '   ')[0])[0]

In [5]:
text = preprocess_text(text)

## 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:

подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);

возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;

расшифруйте их таким частотным методом.


In [6]:
counter = Counter(text)

In [7]:
len(counter)

33

In [8]:
def accuracy(decoded_text, original_text):
    correct_count = 0
    for dec_char, target_char in zip(decoded_text, original_text):
        correct_count += (dec_char == target_char)
    return correct_count / len(decoded_text)

In [9]:
alphabet = sorted(counter.keys())

In [10]:
permutation = ''.join(np.random.permutation(alphabet))

In [11]:
encode_mapping = dict(zip(alphabet, permutation))

In [12]:
def encode_text(text, encode_mapping):
    return ''.join([encode_mapping[char] for char in text])

In [13]:
test_text = "Старик! я слышал много раз, Что ты меня от смерти спас — Зачем?.. Угрюм и одинок, Грозой оторванный листок, Я вырос в сумрачных стенах Душой дитя, судьбой монах. Я никому не мог сказать Священных слов „отец“ и „мать“. Конечно, ты хотел, старик, Чтоб я в обители отвык От этих сладостных имен,— Напрасно: звук их был рожден Со мной. Я видел у других Отчизну, дом, друзей, родных, А у себя не находил Не только милых душ — могил! Тогда, пустых не тратя слез, В душе я клятву произнес: Хотя на миг когда-нибудь Мою пылающую грудь Прижать с тоской к груди другой, Хоть незнакомой, но родной. Увы! теперь мечтанья те Погибли в полной красоте, И я как жил, в земле чужой Умру рабом и сиротой."

In [14]:
test_text = preprocess_text(test_text)
encoded_text = encode_text(test_text, encode_mapping)

In [15]:
test_freq = Counter(encoded_text)

In [16]:
def get_decode_mapping(train_freq, test_freq):
    mapping = {}
    for (char1, _), (char2, _) in zip(train_freq.most_common(), test_freq.most_common()):
        mapping[char2] = char1
    return mapping

In [17]:
decode_mapping = get_decode_mapping(counter, test_freq)

In [18]:
def decode_text(text, decode_mapping):
    return ''.join(decode_mapping[char] for char in text)

In [19]:
decoded_text = decode_text(encoded_text, decode_mapping)

In [20]:
accuracy(decoded_text, test_text)

0.39164086687306504

## 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;

проведите тестирование аналогично п.1, но при помощи биграмм.

In [21]:
def get_bigram_freq(text):
    return Counter([''.join(bigram) for bigram in everygrams(text, max_len=2, min_len=2)])

In [22]:
train_bigram_freq = get_bigram_freq(text)
test_bigram_freq = get_bigram_freq(encoded_text)

In [23]:
decode_mapping = get_decode_mapping(train_bigram_freq, test_bigram_freq)

In [24]:
def decode_text(text, decode_mapping):
    dec_text = []
    for i in range(0, len(text) - 1, 2):
        dec_text.append(decode_mapping[text[i: i + 2]])
    return "".join(dec_text)

In [25]:
decoded_text = decode_text(encoded_text, decode_mapping)

In [26]:
accuracy(decoded_text, test_text)

0.08513931888544891

In [27]:
decoded_text

'л с  ггоо ойейалетй некадотьнек е оти  со у ылстмуатскчттинив ча б ире отов ое п ия удт оса одл еггоноляакноо ськаю воо раро в рсва ревео прик пе рудего н гваеркое й о всесзао ихизнявоо сим  сгл бе  мов дкою нек да сазо ан чтоть агоно и яраод иннты имику ео имтрл ос вулот нымкаеенеутры б вийалпоятото неет пгоношаазнинаенем в слсжеертрорасевжаонелос влеерошр  нь ротареалко квыугненуой в рлылаемаля ивлебрл во нь ще ми лисононаы ь и ичпаая тпоарколотаве нленуны дивт  эпровлапез имопзьтеен з т чд зао  как да тов прстасог пда совкожеедва п ннепосе пнин  кывавову тьт га кь иеемекстм иежн полкамера бгооледьнриносктикиилаш пнивиерках орстахпоя а '

# 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;

реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [28]:
from copy import copy
from tqdm import tqdm

In [59]:
class mcmc_decoder:
    def __init__(self, freqs, alphabet, n_iter=20000, verbose=5000):
        self.n_iter = n_iter
        self.verbose = verbose
        self.freqs = freqs
        self.alphabet = alphabet
        
        self.nan_replacement = 1e-10
        
    def score(self, text):
        bigram_counts = get_bigram_freq("".join(text))
        return np.sum([count * np.log(self.freqs.get(bigram, self.nan_replacement)) 
                       for bigram, count in bigram_counts.items()])
             
    def random_swap(self, text):
        chars = np.random.choice(self.alphabet, 2, replace=False)
        for i in range(len(text)):
            if text[i] == chars[0]:
                text[i] = chars[1]
            elif text[i] == chars[1]:
                text[i] = chars[0]
        return text
        
    def decode(self, text):
        best_decoded_text = list(text)
        cur_score = self.score(best_decoded_text) 
        best_score = cur_score
        for i in tqdm(range(self.n_iter)):
            new_text = self.random_swap(copy(text))
            new_score = self.score(new_text)
            if new_score > cur_score or np.random.rand() < np.exp(new_score - cur_score):
                text = new_text
                cur_score = new_score
                if cur_score > best_score:
                    best_score = cur_score
                    best_decoded_text = text
                    
            if i % self.verbose == 0:
                print(''.join(best_decoded_text))
                print('-' * 80)
                
        return ''.join(best_decoded_text)

In [60]:
train_bigram_freq = get_bigram_freq(text)

In [61]:
mcmc = mcmc_decoder(train_bigram_freq, list(alphabet), n_iter=20000, verbose=5000)

decoded_text = mcmc.decode(list(encoded_text))

  1%|          | 200/20000 [00:00<00:29, 676.16it/s]

йъусбжыщыйзвнузыиодпдысуюы ъдыъвыигощыдъыйигсъбыйхуйыюу гиыкпсеиыбыдабоджыпсдюдьыдъдсцуоовьызбйъджыщыцвсдйыцыйкису овтыйъгоутыакндьыабъщыйкачэдьыидоутыщыобждикыогыидпыйжуюуъчыйцщмгоовтыйздцыдъгшыбыиуъчыждог одыъвытдъгзыйъусбжы ъдэыщыцыдэбъгзбыдъцвжыдъыфъбтыйзуадйъовтыбигоыоухсуйодыюцкжыбтыэвзысдлагоыйдыиодьыщыцбагзыкыаскпбтыдъ бюокыадиыаскюгьысдаовтыуыкыйгэщыогыоутдабзыогыъдзчждыибзвтыакныидпбзыъдпауыхкйъвтыогыъсуъщыйзгюыцыакнгыщыжзщъцкыхсдбюогйытдъщыоуыибпыждпауобэкачыидеыхвзуемкеыпскачыхсблуъчыйыъдйждьыжыпскабыаскпдьытдъчыогюоуждидьыодысдаодьыкцвыъгхгсчыиг ъуочщыъгыхдпбэзбыцыхдзодьыжсуйдъгыбыщыжужылбзыцыюгизгы клдьыкискысуэдиыбыйбсдъдьы
--------------------------------------------------------------------------------


 26%|██▌       | 5176/20000 [00:07<00:21, 697.03it/s]

дтарик я длычал вного раз сто ты веня от дверти дфад засев угрюв и опинок грозой оторманный лидток я мырод м дуврасных дтенах пучой питя дупьбой вонах я никову не вог дказать дмяженных длом отец и вать конесно ты хотел дтарик стоб я м обители отмык от этих длаподтных ивен нафрадно змук их был рошпен до вной я мипел у пругих отсизну пов прузей ропных а у дебя не нахопил не только вилых пуч вогил тогпа фудтых не тратя длез м пуче я клятму фроизнед хотя на виг когпанибупь вою фылаюжую групь фришать д тодкой к групи пругой хоть незнаковой но ропной умы теферь вестанья те фогибли м фолной крадоте и я как шил м зевле сушой увру рабов и диротой 
--------------------------------------------------------------------------------


 51%|█████     | 10170/20000 [00:14<00:13, 711.76it/s]

старик я слышал вного раз что ты веня от сверти спас зачев угрюв и одинок грозой оторманный листок я мырос м суврачных стенах душой дитя судьбой вонах я никову не вог сказать смященных слом отеж и вать конечно ты хотел старик чтоб я м обители отмык от этих сладостных ивен напрасно змук их был роцден со вной я мидел у других отчизну дов друзей родных а у себя не находил не только вилых душ вогил тогда пустых не тратя слез м душе я клятму произнес хотя на виг когданибудь вою пылающую грудь прицать с тоской к груди другой хоть незнаковой но родной умы теперь вечтанья те погибли м полной красоте и я как цил м зевле чуцой увру рабов и сиротой 
--------------------------------------------------------------------------------


 76%|███████▌  | 15157/20000 [00:22<00:06, 700.97it/s]

старик я слышал вного раз что ты веня от сверти спас зачев угрюв и одинок грозой оторманный листок я мырос м суврачных стенах душой дитя судьбой вонах я никову не вог сказать смященных слом отец и вать конечно ты хотел старик чтоб я м обители отмык от этих сладостных ивен напрасно змук их был рожден со вной я мидел у других отчизну дов друзей родных а у себя не находил не только вилых душ вогил тогда пустых не тратя слез м душе я клятму произнес хотя на виг когданибудь вою пылающую грудь прижать с тоской к груди другой хоть незнаковой но родной умы теперь вечтанья те погибли м полной красоте и я как жил м зевле чужой увру рабов и сиротой 
--------------------------------------------------------------------------------


100%|██████████| 20000/20000 [00:28<00:00, 691.05it/s]


In [62]:
accuracy(decoded_text, test_text)

0.9442724458204335

# 4

Расшифруйте сообщение:

In [68]:
def decode_text(text, decode_mapping):
    return ''.join(decode_mapping[char] for char in text)

In [69]:
final_text = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [71]:
unigram_freqs = Counter(final_text)
decode_mapping = get_decode_mapping(counter, unigram_freqs)

decoded_text = decode_text(final_text, decode_mapping)

In [72]:
decoded_text

'олие рд реяесо навгтиундь еие памсе навгтиундь соклс з шсаыа лаабйонеч касавдь иоыка пваместсу лкавоо рлоыа рд рло ляоитие пвтреиуна е паизмесо гтклегтиундь бтии жт палиояноо мосровсао жтятнео кзвлт хасч каномна ч немоыа но абойтю'

In [74]:
mcmc.decode(list(decoded_text))

  2%|▏         | 453/20000 [00:00<00:12, 1518.70it/s]

олие рд реяесо навгтиундь еие памсе навгтиундь соклс з шсаыа лаабйонеч касавдь иоыка пваместсу лкавоо рлоыа рд рло ляоитие пвтреиуна е паизмесо гтклегтиундь бтии жт палиояноо мосровсао жтятнео кзвлт хасч каномна ч немоыа но абойтю
--------------------------------------------------------------------------------


 27%|██▋       | 5398/20000 [00:03<00:09, 1588.86it/s]

если вы вимите норжальный или подти норжальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный чалл за послемнее детвертое замание курса ботя конедно я нидего не очешах
--------------------------------------------------------------------------------


 52%|█████▏    | 10324/20000 [00:06<00:06, 1594.60it/s]

если вы вимите норжальный или подти норжальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный чалл за послемнее детвертое замание курса ботя конедно я нидего не очешах
--------------------------------------------------------------------------------


 77%|███████▋  | 15430/20000 [00:09<00:02, 1589.93it/s]

если вы вимите норжальный или подти норжальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный чалл за послемнее детвертое замание курса ботя конедно я нидего не очешах
--------------------------------------------------------------------------------


100%|██████████| 20000/20000 [00:12<00:00, 1579.24it/s]


'если вы вимите норжальный или подти норжальный текст у этого соочшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный чалл за послемнее детвертое замание курса ботя конедно я нидего не очешах'